In [1]:
import pandas as pd
import numpy as np
import fastavro as fa
import operator
import functools

In [2]:
csr_df = pd.read_csv('./CSR Data.csv')

In [3]:
csr_df = csr_df.rename(columns={'What is your current (pre-CSR) approximate credit score?': 'credit_score', 'Did you apply for the CSR on August 21 or later?': 'apply after aug 21', 'What is your current (pre-CSR) relative 5/24 status? ': 'relative_5/24', 'What is your current (pre-CSR) exact 5/24 status? Put how many cards you\'ve opened in the last 24 months. For instance, if you\'re at 13/24, put just 13.': 'exact_5/24', 'How many Chase cards do you currently (pre-CSR) have? CHASE cards specifically.': 'chase_cards', 'What was your stated income? No commas, put it in format of ######. For instance 46000 or 157000.': 'income', 'What was the INITIAL result of your application? Even if you were eventually approved after pending or denial, put pending or denied for this question so we can capture the relevant information. Only put "instant approval" if you got that right away. ': 'instant_approval', 'What credit limit did you get for the CSR? Put in format of XXXXX. For instance, 10000 or 12000. DON\'T put 10k or 12k, to make it easier to tabulate.': 'credit_limit', 'How long have you been a Chase customer? Any relationship: banking, mortgage, credit cards, etc. Answer in years.': 'years_with_chase', 'How old are you? ': 'age', 'What gender are you?': 'gender'})

# impossible applications
rows_to_drop = [2404, 551, 2192, 2146, 1335, 2317, 547, 387]

for row in rows_to_drop:
    csr_df.drop(row, inplace=True)

invalid_conditions = [csr_df['credit_score'] < 300, csr_df['credit_score'] > 850, csr_df['years_with_chase'] > 16, csr_df['chase_cards'] < 0, csr_df['exact_5/24'] < 0, csr_df['age'] < 16, csr_df['apply after aug 21'] == 'No']
csr_df.drop(csr_df[functools.reduce(operator.or_, invalid_conditions)].index, inplace=True)
csr_df = csr_df[pd.notnull(csr_df['credit_score'])]

cols_to_drop = ['apply after aug 21', 'Timestamp', 'When did you open your last (pre-CSR) Chase card? Again, last CHASE card specifically.', 'How did you apply?', 'When did you apply? See description below for format.', 'What bureau did they pull? Select all that apply.', 'Please provide specifics of the above.', 'Please provide specifics of the above..1', 'If you got a hard denial, have you received the reason (either by letter, or from recon)? Select any/all that apply.', 'Anything else we should know about your CSR application?', 'Do you primarily go for:', 'Which of the following cards do you currently hold? Select all that apply.', 'On average, how much MS do you do PER MONTH?', 'What percentage of your income do you already have as CL with Chase (Pre-CSR)? For instance, if you have 20k CL total among your existing cards on income of 80k, you\'d be at 25%. Put your current % (before applying for CSR) below.', 'How many people are you churning for? ie what is your relationship status?', 'What state do you live in currently? If you don\'t live in the 50 US states or DC, use the "Other" line at the bottom to put in where you are residing.', 'Do you have a checking/savings bank account with Chase? How much is in it? If multiple accounts, use the total.', 'What is the current/final result of your application after it got initially denied? Answer with the most current result in mind - including multiple calls to recon if applicable.', 'What is the current/final result of your application after it initially went pending? Answer with the most current result in mind - including multiple calls to recon if applicable.']

for col in cols_to_drop:
    csr_df.drop(col, axis=1, inplace=True)

In [4]:
# convert string features to numeric values
csr_df.loc[csr_df['gender'] == 'Male', 'gender'] = 0
csr_df.loc[csr_df['gender'] == 'Female', 'gender'] = 1
csr_df.loc[csr_df['gender'].isna(), 'gender'] = 2

csr_df.loc[csr_df['exact_5/24'] >= 5, 'relative_5/24'] = 1
csr_df.loc[csr_df['exact_5/24'] < 5, 'relative_5/24'] = 0

csr_df.loc[csr_df['instant_approval'] == 'Denied', 'instant_approval'] = 0
csr_df.loc[csr_df['instant_approval'] == 'Pending', 'instant_approval'] = 1
csr_df.loc[csr_df['instant_approval'] == 'Instant approval', 'instant_approval'] = 2

csr_df.fillna(0, inplace=True)

csr_df = csr_df.astype({"credit_score": int, "exact_5/24": int, "chase_cards": int, "income": int, "age": int, "years_with_chase": int, "credit_limit": int})

In [5]:
# write to avro database
schema = {
    'doc': 'Chase Sapphire Reserve',
    'name': 'CSR',
    'namespace': 'cc',
    'type': 'record',
    'fields': [
        {'name': 'credit_score', 'type': 'int'},
        {'name': 'relative_5/24', 'type': 'int'},
        {'name': 'exact_5/24', 'type': 'int'},
        {'name': 'chase_cards', 'type': 'int'},
        {'name': 'income', 'type': 'int'},
        {'name': 'age', 'type': 'int'},
        {'name': 'gender', 'type': 'int'},
        {'name': 'years_with_chase', 'type': 'int'},
        {'name': 'instant_approval', 'type': 'int'},
        {'name': 'credit_limit', 'type': 'int'},
    ],
}
parsed_schema = fa.parse_schema(schema)

with open('csr.avro', 'wb') as f:
    fa.writer(f, parsed_schema, csr_df.to_dict('records'), validator=True)